In [1]:
"""import requests
import zipfile


def save_file(url, file_name):
  r = requests.get(url)
  with open(file_name, 'wb') as f:
    f.write(r.content)

    
save_file('https://courses.cs.washington.edu/courses/cse163/19sp/' +
          'files/lectures/05-13/data.zip', 'data.zip')
save_file('https://courses.cs.washington.edu/courses/cse163/19sp/' +
          'files/lectures/05-13/gz_2010_us_040_00_5m.json', 
          'gz_2010_us_040_00_5m.json')
save_file('https://courses.cs.washington.edu/courses/cse163/19sp/' +
          'files/lectures/05-13/stormhistory.csv', 'stormhistory.csv')

with zipfile.ZipFile("data.zip","r") as zip_ref:
    zip_ref.extractall()
    
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely
!pip install --upgrade descartes"""

     |████████████████████████████████| 972kB 2.8MB/s 
     |████████████████████████████████| 10.9MB 18.6MB/s 
     |████████████████████████████████| 14.8MB 293kB/s 
     |████████████████████████████████| 225kB 2.5MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.2-cp36-none-any.whl size=36216 sha256=81894234459bfb2eb8808ef14e892b4e5c3a1b2199ab1aeb18054128f0d39167
  Stored in directory: /root/.cache/pip/wheels/96/6c/53/4112475adf3b831da97f083163d0f38ee6daac9c1b13f7afea
Successfully built pyshp
Requirement already up-to-date: shapely in /usr/local/lib/python3.6/dist-packages (1.7.1)
Requirement already up-to-date: descartes in /usr/local/lib/python3.6/dist-packages (1.1.0)


In [2]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [1]:
%cd E:\yujin\INHA\data

E:\yujin\INHA\data


In [2]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from pandas import Series, DataFrame
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
#gyboundary = gpd.read_file('/gdrive/My Drive/광양시 데이터/20.광양시_행정경계(읍면동).geojson')
gyboundary = gpd.read_file('20.광양시_행정경계(읍면동).geojson')
administrative = gyboundary.ADM_DR_NM.tolist()
gyboundary

,ADM_DR_CD,ADM_DR_NM,geometry
0,3606031,봉강면,"MULTIPOLYGON (((127.58037 35.10902, 127.58052 ..."
1,3606032,옥룡면,"MULTIPOLYGON (((127.59872 35.12006, 127.59919 ..."
2,3606033,옥곡면,"MULTIPOLYGON (((127.64878 35.03547, 127.64902 ..."
3,3606034,진상면,"MULTIPOLYGON (((127.65639 35.11680, 127.65698 ..."
4,3606035,진월면,"MULTIPOLYGON (((127.75715 35.05502, 127.75716 ..."
5,3606036,다압면,"MULTIPOLYGON (((127.64117 35.16850, 127.64140 ..."
6,3606051,골약동,"MULTIPOLYGON (((127.68056 34.92848, 127.68105 ..."
7,3606053,중마동,"MULTIPOLYGON (((127.70376 34.96198, 127.70542 ..."
8,3606054,광영동,"MULTIPOLYGON (((127.71676 34.96939, 127.71691 ..."
9,3606055,태인동,"MULTIPOLYGON (((127.76162 34.95988, 127.76163 ..."


In [4]:
#parkingLot = pd.read_csv('/gdrive/My Drive/광양시 데이터/02.광양시_주차장_공간정보.csv')
parkingLot = pd.read_csv('02.광양시_주차장_공간정보.csv')
parkingLot['lon'] = parkingLot['lon'].astype(float)
parkingLot['lat'] = parkingLot['lat'].astype(float)
parkingLot['geometry'] = parkingLot.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
parkingLot = gpd.GeoDataFrame(parkingLot, geometry='geometry')
parkingLot = parkingLot[['면적','주차면수','geometry']]
parkingLot.rename(columns={"면적":"area"}, inplace=True)

pllist = []
for i in range(len(parkingLot)):
    for j in range(len(gyboundary)):
        if parkingLot.geometry[i].within(gyboundary.geometry[j]) == True:
            pllist.append(gyboundary.ADM_DR_NM[j])

parkingLot['administration'] = pllist
parkingLot

,area,주차면수,geometry,administration
0,4189.0,161.0,POINT (127.58688 34.97032),광양읍
1,2968.0,125.0,POINT (127.69543 34.93625),중마동
2,2836.0,89.0,POINT (127.58459 34.97564),광양읍
3,1980.0,52.0,POINT (127.58068 34.97013),광양읍
4,1970.0,64.0,POINT (127.58071 34.97252),광양읍
...,...,...,...,...
230,264.0,10.0,POINT (127.60482 35.07921),옥룡면
231,1212.0,60.0,POINT (127.61342 35.07067),옥룡면
232,2381.0,96.0,POINT (127.61177 35.00276),옥룡면
233,2090.0,66.0,POINT (127.67472 35.09239),진상면


## 1) 충전기 없는 경우


환경부 2020년 전기자동차 보급 및 충전인프라 구축사업 충전인프라 설치운영지침에 따르면 해당 장소에 기 설치된 충전기가 없는 경우 해당 장소의 주차단위구획 수
× (0.01 + 해당 시‧도의 최근 3년간 신규 차량등록대수 분의 신규 전기차 등록대수) 까지 설치가 가능하다



전라남도청이 공개한 2019년 말 기준 광양시의 지난 3년간의 자동차 등록 현황 대수는 
87,990대(2019년 말) - 78,235대(2016년 9월 기준) = 9755대이다.

In [5]:
#chargingSpot = pd.read_csv('/gdrive/My Drive/광양시 데이터/01.광양시_충전기설치현황.csv')
chargingSpot = pd.read_csv('01.광양시_충전기설치현황.csv')
chargingSpot['lon'] = chargingSpot['lon'].astype(float)
chargingSpot['lat'] = chargingSpot['lat'].astype(float)
chargingSpot['geometry'] = chargingSpot.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
chargingSpot = gpd.GeoDataFrame(chargingSpot, geometry='geometry')
chargingSpot['급속/완속'] = np.where(chargingSpot['급속/완속']=='민간',0,1) #급속 = 0, 완속 = 1
chargingSpot = chargingSpot[['급속/완속','이용대수','geometry']]

cllist = []
for i in range(len(chargingSpot)):
    for j in range(len(gyboundary)):
        if chargingSpot.geometry[i].within(gyboundary.geometry[j]) == True:
            cllist.append(gyboundary.ADM_DR_NM[j])

chargingSpot['administration'] = cllist
chargingSpot

,급속/완속,이용대수,geometry,administration
0,1,2,POINT (127.56830 34.96300),광양읍
1,1,1,POINT (127.58332 34.98033),광양읍
2,1,5,POINT (127.68901 34.95376),중마동
3,1,2,POINT (127.71671 34.96403),광영동
4,1,2,POINT (127.73244 34.94044),금호동
5,1,1,POINT (127.58244 34.96333),광양읍
6,1,2,POINT (127.75098 34.96251),진월면
7,1,1,POINT (127.76007 34.94120),태인동
8,1,2,POINT (127.72513 34.96215),광영동
9,1,1,POINT (127.58061 34.97306),광양읍


In [6]:
cslist = []
for i in range(len(gyboundary)):
    a = 0
    for j in range(len(chargingSpot)):
        if chargingSpot.administration[j] == gyboundary.ADM_DR_NM[i]:
            a +=  1
    cslist.append(a)
            

print(cslist) 

[0, 0, 1, 0, 5, 0, 0, 16, 6, 2, 1, 13]


In [7]:
data = {'administration': administrative, 'supply' : cslist}
chargsup  = pd.DataFrame(data)
non = chargsup.supply == 0
nonCharging = chargsup[non]
nonCharging

,administration,supply
0,봉강면,0
1,옥룡면,0
3,진상면,0
5,다압면,0
6,골약동,0


In [8]:
#ElecCar = pd.read_csv('/gdrive/My Drive/광양시 데이터/06.광양시_전기차보급현황(연도별,읍면동별).csv')
ElecCar = pd.read_csv('06.광양시_전기차보급현황(연도별,읍면동별).csv')
adm = ElecCar.행정구역.unique()

In [9]:
#ElecCar = pd.read_csv('/gdrive/My Drive/광양시 데이터/06.광양시_전기차보급현황(연도별,읍면동별).csv')
x = []
y = []
z = []
q = []
for i in range(len(ElecCar)):
    if ElecCar.기준년도[i] == 2017:
        x.append(ElecCar.보급현황[i])
    elif ElecCar.기준년도[i] == 2018:
        y.append(ElecCar.보급현황[i])
    elif ElecCar.기준년도[i] == 2019:
        z.append(ElecCar.보급현황[i])
    else:
        q.append(ElecCar.보급현황[i])


data = {'administration': adm, '2017' : x, '2018' : y , "2019" : z, '2020': q}
ElecCar = pd.DataFrame(data)
ElecCar['2017'].astype('int')
ElecCar['2018'].astype('int')
ElecCar['2019'].astype('int')
ElecCar['2020'].astype('int')
ElecCar

,administration,2017,2018,2019,2020
0,광양읍,17,1,44,40
1,봉강면,0,0,4,0
2,옥룡면,3,0,3,5
3,옥곡면,3,0,2,3
4,진상면,1,0,2,2
5,진월면,1,0,2,2
6,다압면,0,0,0,0
7,골약동,2,1,1,0
8,중마동,17,8,27,26
9,광영동,3,0,6,3


In [10]:
elist = []
for i in range(len(ElecCar)):
    a = 0
    for j in range(1, len(ElecCar.columns)-1):
        if ElecCar.loc[i][j] < ElecCar.loc[i][j+1]:
            x = ElecCar.loc[i][j+1] - ElecCar.loc[i][j]
            a += x
    elist.append(a)

print(elist) 

[43, 4, 5, 3, 2, 2, 0, 0, 19, 6, 0, 9]


In [11]:
administrative.remove('광양읍')
administrative.insert(0, '광양읍')
administrative

['광양읍',
 '봉강면',
 '옥룡면',
 '옥곡면',
 '진상면',
 '진월면',
 '다압면',
 '골약동',
 '중마동',
 '광영동',
 '태인동',
 '금호동']

In [12]:
data = {'administration': administrative, 'supply' : elist}
ElecCar_Supply = pd.DataFrame(data)
total = sum(ElecCar_Supply.supply) # 총 전기차 합계
ElecCar_Supply

,administration,supply
0,광양읍,43
1,봉강면,4
2,옥룡면,5
3,옥곡면,3
4,진상면,2
5,진월면,2
6,다압면,0
7,골약동,0
8,중마동,19
9,광영동,6


### 식 대입해서 계산
해당 장소의 주차단위구획 수 × (0.01 + 해당 시‧도의 최근 3년간 신규 차량등록대수 분의 신규 전기차 등록대수) 의 값이 1 이상인 경우


In [13]:
non = nonCharging.administration.tolist()
new_chargeparking = parkingLot[parkingLot.administration.isin(non)]
new_chargeparking

,area,주차면수,geometry,administration
92,530.0,18.0,POINT (127.68316 34.93816),골약동
93,2363.0,80.0,POINT (127.65790 34.90943),골약동
94,5440.0,220.0,POINT (127.66669 34.91149),골약동
95,116.0,4.0,POINT (127.64741 34.92059),골약동
128,1800.0,58.0,POINT (127.60508 35.04139),옥룡면
129,495.0,20.0,POINT (127.61726 35.01384),옥룡면
142,460.0,36.0,POINT (127.71404 34.99157),진상면
143,397.0,17.0,POINT (127.71928 35.01842),진상면
144,1530.0,46.0,POINT (127.67434 35.09507),진상면
145,1122.0,20.0,POINT (127.71287 35.00518),진상면


In [14]:
parkingArea = new_chargeparking.주차면수.tolist()

print(parkingArea)

[18.0, 80.0, 220.0, 4.0, 58.0, 20.0, 36.0, 17.0, 46.0, 20.0, 8.0, 68.0, 50.0, 73.0, 40.0, 100.0, 100.0, 52.0, 15.0, 30.0, 135.0, 80.0, 10.0, 60.0, 96.0, 66.0, 30.0]


In [15]:
non_chargeparking = new_chargeparking[['area', '주차면수', 'geometry', 'administration']]
palist = []
for i in parkingArea:
    x = i * (0.01 + (total/9755))
    palist.append(x)

non_chargeparking['weight'] = palist
non = non_chargeparking['weight'] >= 1
non_chargeparking = non_chargeparking[non]
non_chargeparking

,area,주차면수,geometry,administration,weight
93,2363.0,80.0,POINT (127.65790 34.90943),골약동,1.562686
94,5440.0,220.0,POINT (127.66669 34.91149),골약동,4.297386
128,1800.0,58.0,POINT (127.60508 35.04139),옥룡면,1.132947
153,1251.0,68.0,POINT (127.57241 35.08692),봉강면,1.328283
155,3031.0,73.0,POINT (127.56466 35.07915),봉강면,1.425951
223,3389.0,100.0,POINT (127.56412 35.07862),봉강면,1.953357
224,3210.0,100.0,POINT (127.57241 35.08692),봉강면,1.953357
225,NaN,52.0,POINT (127.56726 35.08124),봉강면,1.015746
228,4702.0,135.0,POINT (127.59929 35.10153),옥룡면,2.637032
229,1603.0,80.0,POINT (127.59639 35.09324),옥룡면,1.562686


## 2) 설치된 충전소가 있는 경우

In [16]:
충전기 = pd.read_csv("01.광양시_충전기설치현황.csv")
충전기

,충전소명,충전소위치,충전기 운영기관,급속/완속,충전기용량,이용대수,충전기타입,요금정보(원/kw),lon,lat
0,LF스퀘어 광양점,전남 광양시 광양읍 순광로 466 1층,환경부(한국자동차환경협회),급속,200KW 동시충전,2,DC콤보,255.70,127.568300,34.963002
1,광양만권경제자유구역청,전남 광양시 광양읍 인덕로 1100(광양시 제2청사),환경부(한국자동차환경협회),급속,50KW,1,DC차데모/AC3상/DC콤보,255.70,127.583323,34.980334
2,광양송보파인빌5차아파트,전남 광양시 진등길 55-5 (마동 송보파인빌류 아파트) 501동~506동 상가동,케이티,완속,7kw/h,5,AC완속,170.00,127.689013,34.953759
3,광양시 모리스모텔,전남 광양시 광영로 118-1 지상주차장,차지비,완속,7kw/h,2,AC완속,250.00,127.716708,34.964032
4,광양시 백운쇼핑센타,전남 광양시 폭포사랑길 99 주차장,차지비,완속,7kw/h,2,AC완속,245.00,127.732440,34.940444
5,광양시 블루핸즈(광양정비센터),전남 광양시 광양읍 해광로 888 실외주차장,차지비,완속,7kw/h,1,AC완속,245.00,127.582443,34.963334
6,광양시 비치모텔,전남 광양시 진월면 백운1로 389 주차장,차지비,완속,7kw/h,2,AC완속,250.00,127.750981,34.962506
7,광양시 포스코 ICT 광양사업소,전남 광양시 태인4길 20 주차장,차지비,완속,7kw/h,1,AC완속,250.00,127.760072,34.941198
8,광양시 프라자모텔,전남 광양시 광영로 34,차지비,완속,7kw/h,2,AC완속,250.00,127.725127,34.962147
9,광양시 광양읍사무소,전남 광양시 광양읍 남등길 6,한국전기차충전서비스,완속,7kw/h,1,AC완속,255.70,127.580612,34.973057


In [17]:
주차장 = pd.read_csv('02.광양시_주차장_공간정보.csv')
주차장

,주차장명칭,구분,면적,주차면수,유료/무료,주소,lon,lat
0,인동숲앞 주차장,노외,4189.0,161.0,유료,인서리 237-1,127.586883,34.970324
1,구터미널주차장,노외,2968.0,125.0,유료,중동 1651,127.695428,34.936252
2,구경찰서 주차장,노외,2836.0,89.0,무료,읍내리 227-1,127.584586,34.975637
3,칠성1지구,노외,1980.0,52.0,무료,칠성리 962-1,127.580683,34.970130
4,칠성1지구,노외,1970.0,64.0,무료,칠성리 948-1,127.580707,34.972523
...,...,...,...,...,...,...,...,...
230,포석정위도로변,피서지,264.0,10.0,무료,동곡리 662-1,127.604822,35.079214
231,동곡산장옆,피서지,1212.0,60.0,무료,동곡리 549,127.613417,35.070673
232,하천첨주변,피서지,2381.0,96.0,무료,산남리 154-4,127.611774,35.002759
233,회계분교,피서지,2090.0,66.0,무료,진상 어치 1140,127.674719,35.092388


In [32]:
#주차장 데이터에 현재 설치된 충전기가 포함되어 있는지 살펴보았는데 없음.
#1번 공식 그대로 적용

충전기_주차장 = []
for i in range(len(충전기)):
    for j in range(len(주차장)):
        if (충전기['lon'][i] == 주차장['lon'][j]) and (충전기['lat'][i] == 주차장['lat'][j]):
            충전기_주차장.append(j)
충전기_주차장

[]

In [33]:
#3년간 신규 전기차 등록대수
ElecCar_Supply

,administration,supply
0,광양읍,43
1,봉강면,4
2,옥룡면,5
3,옥곡면,3
4,진상면,2
5,진월면,2
6,다압면,0
7,골약동,0
8,중마동,19
9,광영동,6


In [40]:
ElecCar_Supply['supply'].sum()

93

In [37]:
parkingLot

,area,주차면수,geometry,administration
0,4189.0,161.0,POINT (127.58688 34.97032),광양읍
1,2968.0,125.0,POINT (127.69543 34.93625),중마동
2,2836.0,89.0,POINT (127.58459 34.97564),광양읍
3,1980.0,52.0,POINT (127.58068 34.97013),광양읍
4,1970.0,64.0,POINT (127.58071 34.97252),광양읍
...,...,...,...,...
230,264.0,10.0,POINT (127.60482 35.07921),옥룡면
231,1212.0,60.0,POINT (127.61342 35.07067),옥룡면
232,2381.0,96.0,POINT (127.61177 35.00276),옥룡면
233,2090.0,66.0,POINT (127.67472 35.09239),진상면


In [44]:
#해당 장소의 주차단위구획 수 × (0.01 + 해당 시‧도의 최근 3년간 신규 차량등록대수 분의 신규 전기차 등록대수)

weight_lst = []
for i in parkingLot['주차면수']:
    x = i * (0.01 + (ElecCar_Supply['supply'].sum()/9755))
    weight_lst.append(x)

parkingLot['weight'] = weight_lst
parkingLot

,area,주차면수,geometry,administration,weight
0,4189.0,161.0,POINT (127.58688 34.97032),광양읍,3.144905
1,2968.0,125.0,POINT (127.69543 34.93625),중마동,2.441697
2,2836.0,89.0,POINT (127.58459 34.97564),광양읍,1.738488
3,1980.0,52.0,POINT (127.58068 34.97013),광양읍,1.015746
4,1970.0,64.0,POINT (127.58071 34.97252),광양읍,1.250149
...,...,...,...,...,...
230,264.0,10.0,POINT (127.60482 35.07921),옥룡면,0.195336
231,1212.0,60.0,POINT (127.61342 35.07067),옥룡면,1.172014
232,2381.0,96.0,POINT (127.61177 35.00276),옥룡면,1.875223
233,2090.0,66.0,POINT (127.67472 35.09239),진상면,1.289216
